In [1]:
from scipy.linalg import orthogonal_procrustes
import random
import numpy as np
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import splu
import matplotlib.pyplot as plt
from scipy.sparse.linalg import spsolve
import time

def generate_matrix(k, n):
    matrix = np.zeros((n, n))
    
    for i in range(n):
        s = 0
        for j in range(n):
            if i != j:
                matrix[i][j] = random.choice([-1, -2, -3, -4])
                s += matrix[i][j]
        if i == 0:
            matrix[i][i] = -s + 10**(-k)
        else:
            matrix[i][i] = -s
            
    return matrix

def hilbert_matrix(n):
    matrix = np.zeros((n, n))
    
    for i in range(1, n+1):
        for j in range(1, n+1):
            matrix[i-1][j-1] = 1 / (i + j - 1)
    
    return matrix

def solve_gauss(A, b):
    n = A.shape[0]
    Ab = np.hstack((A, b[:, np.newaxis]))
    ops = 0 

    for i in range(n):
        max_row = np.argmax(np.abs(Ab[i:, i])) + i
        
        # Обменять строки
        Ab[[i, max_row]] = Ab[[max_row, i]]
        
        # Элиминация
        for j in range(i+1, n):
            factor = Ab[j, i] / Ab[i, i]
            Ab[j] -= factor * Ab[i]
            ops += 2*n 
    
    x = np.zeros(n)
    
    # Обратная подстановка
    for i in range(n-1, -1, -1):
        x[i] = (Ab[i, -1] - np.dot(Ab[i, :-1], x)) / Ab[i, i]
        ops += 2*n 

    return x, ops

def lu_algorithm(A):
    n = A.shape[0]
    L = np.zeros((n, n))
    U = np.zeros((n, n))
    ops = 0
    
    for k in range(n):
        L[k, k] = 1.0
        U[k, k] = A[k, k] - L[k, :k] @ U[:k, k]
        ops += 1

        for j in range(k+1, n):
            U[k, j] = A[k, j] - L[k, :k] @ U[:k, j]
            ops += 2
        
        for i in range(k+1, n):
            L[i, k] = (A[i, k] - L[i, :k] @ U[:k, k]) / U[k, k]
            ops += 3
    
    return L, U, ops

def lu_solve(A, b):
    L, U, ops = lu_algorithm(A)
    n = A.shape[0]
    x = np.zeros(n)
    y = np.zeros(n)
    
    for i in range(n):
        y[i] = b[i] - L[i, :i] @ y[:i]
        ops += 2
    
    for i in range(n-1, -1, -1):
        x[i] = (y[i] - U[i, i+1:] @ x[i+1:]) / U[i, i]
        ops += 3
    
    return x, ops


def jacobi(A, b, x0, eps=1e-6, max_iter=10000):
    n = len(A)
    x = x0.copy()
    residual = np.inf
    iterations = 0
    
    while residual > eps and iterations < max_iter:
        x_new = np.zeros_like(x)
        
        for i in range(n):
            x_new[i] = (b[i] - np.dot(A[i, :i].tolist(), x_new[:i].tolist()) - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]
        
        residual = np.linalg.norm(x_new - x)
        x = x_new.tolist()
        iterations += 1
    
    return x, iterations

def general_test(A, x, b):
  x_hat, iter = solve_gauss(A, b)
  print("Методом Гаусса: ")
  print_result(x, x_hat)

  x_hat, iter = lu_solve(A, b)
  print("Методом LU-разложения: ")
  print_result(x, x_hat)

  x_hat, iter = jacobi(A, b, np.zeros(len(A)))
  print("Методом Якоби: ")
  print_result(x, x_hat)


def print_result(x, x_approx):
  print("Точное решение: ", x)
  print("Приближенное решение: ", x_approx)
  print("Норма разности: ", np.linalg.norm(x - x_approx))
  print()
  
def print_matrix(A):
  print()
  print(A)
  print("k: ", np.linalg.cond(A))
  print()    

A = generate_matrix(1, 5)
x = np.array([i+1 for i in range(5)])
b = np.dot(A, x)

print("Для матрицы A(k):")
print_matrix(A)
general_test(A, x, b)

A = hilbert_matrix(5)
x = np.array([i+1 for i in range(5)])
b = np.dot(A, x)

print("Для матрицы Гилберта:")
print_matrix(A)
general_test(A, x, b)

Для матрицы A(k):

[[ 9.1 -3.  -1.  -1.  -4. ]
 [-4.  13.  -4.  -4.  -1. ]
 [-1.  -1.   8.  -3.  -3. ]
 [-2.  -4.  -2.  11.  -3. ]
 [-3.  -1.  -1.  -4.   9. ]]
k:  827.0180784061495

Методом Гаусса: 
Точное решение:  [1 2 3 4 5]
Приближенное решение:  [1. 2. 3. 4. 5.]
Норма разности:  1.150930250234631e-13

Методом LU-разложения: 
Точное решение:  [1 2 3 4 5]
Приближенное решение:  [1. 2. 3. 4. 5.]
Норма разности:  1.812271696422804e-13

Методом Якоби: 
Точное решение:  [1 2 3 4 5]
Приближенное решение:  [0.999896091778329, 1.9998952287491354, 2.9998951706720187, 3.999895344464854, 4.999895561401728]
Норма разности:  0.00023371633102811995

Для матрицы Гилберта:

[[1.         0.5        0.33333333 0.25       0.2       ]
 [0.5        0.33333333 0.25       0.2        0.16666667]
 [0.33333333 0.25       0.2        0.16666667 0.14285714]
 [0.25       0.2        0.16666667 0.14285714 0.125     ]
 [0.2        0.16666667 0.14285714 0.125      0.11111111]]
k:  476607.2502422687

Методом Гаусса

In [2]:
def test_diag(n, k_values):
  cond_values = np.zeros(len(k_values))
  rel_err_gauss = np.zeros(len(k_values))
  rel_err_lu = np.zeros(len(k_values))
  rel_err_jacobi = np.zeros(len(k_values))

  print("Для n =", n, " и k в", k_values)

  for i, k in enumerate(k_values):
    A = generate_matrix(k, n)
    cond_values[i] = np.linalg.cond(A)

    x = np.array([i+1 for i in range(n)])
    b = np.dot(A, x)

    x_hat, iter = solve_gauss(A, b)
    rel_err_gauss[i] = np.linalg.norm(x - x_hat)

    x_hat, iter = lu_solve(A, b)
    rel_err_lu[i] = np.linalg.norm(x - x_hat)

    x_hat, iter = jacobi(A, b, np.zeros(n))
    rel_err_jacobi[i] = np.linalg.norm(x - x_hat)

  print()
  print("Число обусловленности: ", cond_values)
  print()
  print("Точность методом Гаусса: ", rel_err_gauss)
  print("Точность методом LU-разложения: ", rel_err_lu)
  print("Точность методом Якоби: ", rel_err_jacobi)
  print()

def test_hilbert(n_values):
  cond_values = np.zeros(len(n_values))
  rel_err_gauss = np.zeros(len(n_values))
  rel_err_lu = np.zeros(len(n_values))
  rel_err_jacobi = np.zeros(len(n_values))

  print("Для n =", n_values)
  for i, n in enumerate(n_values):
    A = hilbert_matrix(n)
    cond_values[i] = np.linalg.cond(A)

    x = np.array([i+1 for i in range(n)])
    b = np.dot(A, x)

    x_hat, iter = solve_gauss(A, b)
    rel_err_gauss[i] = np.linalg.norm(x - x_hat)

    x_hat, iter = lu_solve(A, b)
    rel_err_lu[i] = np.linalg.norm(x - x_hat)

    x_hat, iter = jacobi(A, b, np.zeros(n))
    rel_err_jacobi[i] = np.linalg.norm(x - x_hat)
  
  print()
  print("Число обусловленности: ", cond_values)
  print()
  print("Точность методом Гаусса: ", rel_err_gauss)
  print("Точность методом LU-разложения: ", rel_err_lu)
  print("Точность методом Якоби: ", rel_err_jacobi)
  print()


print()
print("Исследуем зависимость числа обусловленности и точности от параметра k: ")
print()
test_diag(5, [1, 2, 3, 4, 5])
test_diag(10,  [1, 2, 3, 4, 5])
test_diag(15,  [1, 2, 3, 4, 5])
test_diag(20,  [1, 2, 3, 4, 5])

print()
print("Исследуем зависимость числа обусловленности и точности от параметра размера матрицы Гилберта: ")
print()
test_hilbert([2, 3, 4, 5, 6])


Исследуем зависимость числа обусловленности и точности от параметра k: 

Для n = 5  и k в [1, 2, 3, 4, 5]

Число обусловленности:  [1.23578679e+03 7.23747128e+03 1.07317162e+05 1.02933224e+06
 1.17187543e+07]

Точность методом Гаусса:  [1.91852706e-13 2.28010845e-12 2.57165217e-11 2.48500874e-10
 1.99840144e-15]
Точность методом LU-разложения:  [4.60365246e-13 8.30110073e-13 4.26956157e-12 4.51817244e-11
 2.01848030e-09]
Точность методом Якоби:  [3.68544255e-04 1.51191231e-01 6.18660296e+00 8.83573436e+00
 8.55452323e+00]

Для n = 10  и k в [1, 2, 3, 4, 5]

Число обусловленности:  [2.77691466e+03 3.57380443e+04 5.28245206e+05 3.02724338e+06
 2.17978474e+07]

Точность методом Гаусса:  [2.03383100e-12 1.15142355e-11 1.18206252e-10 7.86035134e-10
 7.90250672e-09]
Точность методом LU-разложения:  [1.48312001e-12 9.16566315e-12 2.22108386e-10 1.57206177e-09
 1.08390236e-08]
Точность методом Якоби:  [9.90659435e-04 9.77727852e+00 2.19110216e+01 2.30098233e+01
 2.27181217e+01]

Для n = 15  и

In [3]:
def test_iter_by_n(n_values, n_matrix):
  for i,n in enumerate(n_values):
    print("Для n = ", n)
    print()

    A = n_matrix[i]
    x = np.arange(1, n + 1)
    b = np.dot(A, x)

    start_jacobi = time.time()
    x_approx_jacobi, jacobi_iter = jacobi(A, b, np.zeros(n))
    time_jacobi = time.time() - start_jacobi

    start_gauss = time.time()
    x_approx_gauss, gauss_iter = solve_gauss(A, b)
    time_gauss = time.time() - start_gauss
    
    start_lu = time.time()
    x_approx_lu, lu_iter = lu_solve(A, b)
    time_lu = time.time() - start_lu

    print("Колличество операций методом Гаусса: ", gauss_iter)
    print("Колличество операций методом LU-разложения: ", lu_iter)
    print("Колличество итераций методом Якоби: ", jacobi_iter)
    print()
    print("Время методом Гаусса: ", time_gauss, "сек")
    print("Время методом LU-разложения: ", time_lu, "сек")
    print("Время методом Якоби: ", time_jacobi, "сек")
    print()
    print("Норма разности методом Гаусса: ", np.linalg.norm(x - x_approx_gauss))
    print("Норма разности методом LU-разложения: ", np.linalg.norm(x - x_approx_lu))
    print("Норма разности методом Якоби: ", np.linalg.norm(x - x_approx_jacobi))
    print()

def generate_n_matrix(n_values):
  n_diag_matrix = []
  n_hilbert_matrix = []
  for i,n in enumerate(n_values):
    n_diag_matrix.append(generate_matrix(5, n))
    n_hilbert_matrix.append(hilbert_matrix(n))

  return n_diag_matrix, n_hilbert_matrix

n_diag_matrix, n_hilbert_matrix = generate_n_matrix([10, 50, 100, 1000])

print("Эффективность для матриц A(k)")
print()
test_iter_by_n([10, 50, 100, 1000], n_diag_matrix)

print("Эффективность для матриц Гилберта")
print()
test_iter_by_n([10, 50, 100, 1000], n_hilbert_matrix)

Эффективность для матриц A(k)

Для n =  10
Колличество операций методом Гаусса:  1100
Колличество операций методом LU-разложения:  285
Колличество итераций методом Якоби:  10000

Время методом Гаусса:  0.0010035037994384766 сек
Время методом LU-разложения:  0.0 сек
Время методом Якоби:  0.5885467529296875 сек

Норма разности методом Гаусса:  3.335559088943214e-08
Норма разности методом LU-разложения:  1.5543122344752192e-15
Норма разности методом Якоби:  23.36751363432133

Для n =  50

Колличество операций методом Гаусса:  127500
Колличество операций методом LU-разложения:  6425
Колличество итераций методом Якоби:  11

Время методом Гаусса:  0.004000425338745117 сек
Время методом LU-разложения:  0.006005525588989258 сек
Время методом Якоби:  0.004014730453491211 сек

Норма разности методом Гаусса:  6.651558314897401e-06
Норма разности методом LU-разложения:  1.1194207015080788e-06
Норма разности методом Якоби:  242.4043721459442

Для n =  100

Колличество операций методом Гаусса:  1010

KeyboardInterrupt: 